![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/01.2.Resume_MedicalNer_Model_Training.ipynb)

# Resume MedicalNer Model Training

Steps:
- Train a new model for a few epochs.
- Load the same model and train for more epochs on the same taxnonomy, and check stats.
- Train a model already trained on a different data

## Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs==5.1.0

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [ ]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

In [ ]:
spark

## Download Clinical Word Embeddings for training

In [ ]:
clinical_embeddings = nlp.WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


## Download Data for Training (NCBI Disease Dataset)

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NCBI_disease_official_test.conll
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NCBI_disease_official_train_dev.conll

In [ ]:
# from sparknlp.training import CoNLL

training_data = nlp.CoNLL().readDataset(spark, 'NCBI_disease_official_train_dev.conll')

training_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Identification of...|[{document, 0, 89...|[{document, 0, 89...|[{token, 0, 13, I...|[{pos, 0, 13, NN,...|[{named_entity, 0...|
|The adenomatous p...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
|Complex formation...|[{document, 0, 63...|[{document, 0, 63...|[{token, 0, 6, Co...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [ ]:
test_data = nlp.CoNLL().readDataset(spark, 'NCBI_disease_official_test.conll')

test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Clustering of mis...|[{document, 0, 10...|[{document, 0, 10...|[{token, 0, 9, Cl...|[{pos, 0, 9, NN, ...|[{named_entity, 0...|
|Ataxia - telangie...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 5, At...|[{pos, 0, 5, NN, ...|[{named_entity, 0...|
|The risk of cance...|[{document, 0, 15...|[{document, 0, 15...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



## Split the test data into two parts:
- We Keep the first part separate and use it for training the model further, as it will be totally unseen data from the same taxonomy.

- The second part will be used to testing and evaluating

In [ ]:
(test_data_1, test_data_2) = test_data.randomSplit([0.5, 0.5], seed = 100)

# save the test data as parquet for easy testing
clinical_embeddings.transform(test_data_1).write.parquet('test_1.parquet')

clinical_embeddings.transform(test_data_2).write.parquet('test_2.parquet')

## Train a new model, pause, and resume training on the same dataset.

### Create a graph

We will use `TFGraphBuilder` annotator which can be used to create graphs in the model training pipeline. `TFGraphBuilder` inspects the data and creates the proper graph if a suitable version of TensorFlow (<= 2.7 ) is available. The graph is stored in the defined folder and loaded by the approach.

In [ ]:
!pip install -q tensorflow==2.12.0
!pip install -q tensorflow-addons

In [ ]:
graph_folder_path = "medical_ner_graphs"

ner_graph_builder = medical.TFGraphBuilder()\
    .setModelName("ner_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder_path)\
    .setGraphFile("auto")\
    .setHiddenUnitsNumber(24)\
    .setIsLicensed(True) # False -> if you want to use TFGraphBuilder with NerDLApproach

In [ ]:
# You can create custom TensorFlow graph file (`.pb` extension) for NER model training externally by using the script below.
# If this method is used, graph folder should be added to NerApproach as  `.setGraphFolder(graph_folder_path)` .

'''
medical.tf_graph.print_model_params("ner_dl")

graph_folder_path = "medical_ner_graphs"

medical.tf_graph.build("ner_dl",
               build_params={"embeddings_dim": 200,
                             "nchars": 128,
                             "ntags": 12,
                             "is_medical": 1},
                model_location=graph_folder_path,
                model_filename="auto")
'''

### Train for 2 epochs

In [ ]:
nerTagger = medical.NerApproach()\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(2)\
    .setLr(0.003)\
    .setBatchSize(8)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setTestDataset('./test_2.parquet')\
    .setGraphFolder(graph_folder_path)\
    .setOutputLogsPath('./ner_logs')

ner_pipeline = nlp.Pipeline(stages=[
    clinical_embeddings,
    ner_graph_builder,
    nerTagger
 ])

In [ ]:
%%time
ner_model = ner_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 13.4 s, sys: 690 ms, total: 14.1 s
Wall time: 2min 13s


In [ ]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_722d2eb14883.log


In [ ]:
# Training Logs
! cat ner_logs/MedicalNerApproach*

Name of the selected graph: /content/medical_ner_graphs/blstm_3_200_24_85.pb
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 6347


Epoch 1/2 started, lr: 0.003, dataset size: 6347


Epoch 1/2 - 43.31s - loss: 2528.7737 - avg training loss: 3.1808474 - batches: 795
Quality on test dataset: 
time to finish evaluation: 1.80s
Total test loss: 97.6228	Avg test loss: 1.5254
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 456	 50	 174	 0.90118575	 0.72380954	 0.8028169
B-Disease	 444	 90	 87	 0.83146065	 0.8361582	 0.83380276
tp: 900 fp: 140 fn: 261 labels: 2
Macro-average	 prec: 0.86632323, rec: 0.7799839, f1: 0.82088953
Micro-average	 prec: 0.86538464, rec: 0.7751938, f1: 0.81781006


Epoch 2/2 started, lr: 0.0029850747, dataset size: 6347


Epoch 2/2 - 39.90s - loss: 1077.7648 - avg training loss: 1.3556789 - batches: 795
Quality on test dataset: 
time to finish evaluation: 0.97s
Total test loss: 86.3864	Avg test loss: 1.3498
label	 

In [ ]:
# Logs of 4 consecutive epochs to compare with 2+2 epochs on separate datasets from same taxonomy

#!cat ner_logs/MedicalNerApproach_4d3d69967c3f.log

### Evaluate

In [ ]:
#from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+-----+-----+---------+------+------+
| entity|   tp|  fp|   fn|total|precision|recall|    f1|
+-------+-----+----+-----+-----+---------+------+------+
|Disease|366.0|66.0|161.0|527.0|   0.8472|0.6945|0.7633|
+-------+-----+----+-----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.7632950990615224|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.7632950990615224|
+------------------+

None


`ner_utils`: This new module is used after NER training to calculate mertic chunkbase and plot training logs.

`evaluate`: if verbose, returns overall performance, as well as performance per chunk type; otherwise, simply returns overall precision, recall, f1 scores

`loss_plot`: Plots the figure of loss vs epochs

`get_charts` : Plots the figures of metrics ( precision, recall, f1) vs epochs

```
from sparknlp_jsl.utils.ner_utils import get_charts, loss_plot, evaluate

metrics = evaluate(preds_df['prediction'].values, preds_df['ground_truth'].values)

loss_plot(f"./ner_logs/{log_file}")

get_charts('./ner_logs/'+log_files[0])
```

### Save the model to disk

In [ ]:
ner_model.stages[2].write().overwrite().save('models/NCBI_NER_2_epoch')

### Train using the saved model on unseen dataset
#### We use unseen data from same taxonomy

In [ ]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(2)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('/content/test_2.parquet')\
      .setOutputLogsPath('ner_logs')\
      .setGraphFolder(graph_folder_path)\
      .setPretrainedModelPath("models/NCBI_NER_2_epoch") ## load existing model

ner_pipeline = nlp.Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [ ]:
# remove the existing logs

! rm -r ./ner_logs

In [ ]:
%%time
ner_model_retrained = ner_pipeline.fit(test_data_1)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 79, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_79.pb
CPU times: user 9.67 s, sys: 44.2 ms, total: 9.71 s
Wall time: 26.9 s


In [ ]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_4df6d2167a8d.log


In [ ]:
!cat ./ner_logs/MedicalNerApproach*

Name of the selected graph: pretrained graph
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 438


Epoch 1/2 started, lr: 0.003, dataset size: 438


Epoch 1/2 - 5.86s - loss: 76.7992 - avg training loss: 1.3473544 - batches: 57
Quality on test dataset: 
time to finish evaluation: 1.46s
Total test loss: 65.7469	Avg test loss: 1.0273
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 500	 54	 130	 0.9025271	 0.7936508	 0.8445946
B-Disease	 447	 66	 84	 0.87134504	 0.8418079	 0.8563219
tp: 947 fp: 120 fn: 214 labels: 2
Macro-average	 prec: 0.88693607, rec: 0.81772935, f1: 0.8509279
Micro-average	 prec: 0.88753515, rec: 0.81567615, f1: 0.8500898


Epoch 2/2 started, lr: 0.0029850747, dataset size: 438


Epoch 2/2 - 3.14s - loss: 59.122223 - avg training loss: 1.0372319 - batches: 57
Quality on test dataset: 
time to finish evaluation: 0.93s
Total test loss: 65.1022	Avg test loss: 1.0172
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 515	 43

In [ ]:
#from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

pred_df = ner_model_retrained.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label",case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|459.0|93.0|68.0|527.0|   0.8315| 0.871|0.8508|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8507877664504171|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8507877664504171|
+------------------+

None


### Train a Model Choosing Best Model

`useBestModel`:  This param preserves and restores the model that has achieved the best performance at the end of the training. The priority is metrics from testDataset (micro F1), metrics from validationSplit (micro F1), and if none is set it will keep track of loss during the training. <br/>

`earlyStopping`: You can stop training at the point when the perforfmance on test/validation dataset starts to degrage. Two params are used in order to use this feature: <br/>

- `earlyStoppingCriterion`: This is used set the minimal improvement of the test metric to terminate training. The metric monitored is the same as the metrics used in `useBestModel` (micro F1 when using test/validation set, loss otherwise). Default is 0 which means no early stopping is applied.

- `earlyStoppingPatience`:  The number of epoch without improvement which will be tolerated. Default is 0, which means that early stopping will occur at the first time when performance in the current epoch is no better than in the previous epoch.

In [ ]:
# remove the existing logs

! rm -r ./ner_logs

In [ ]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(20)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('./test_2.parquet')\
      .setGraphFolder(graph_folder_path)\
      .setOutputLogsPath('./ner_logs')\
      .setValidationSplit(0.2)\
      .setUseBestModel(True)\
      .setEarlyStoppingCriterion(0.04)\
      .setEarlyStoppingPatience(3)


ner_pipeline = nlp.Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [ ]:
%%time
ner_model = ner_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 10.1 s, sys: 119 ms, total: 10.2 s
Wall time: 2min 53s


In [ ]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_e2b0cb32afb0.log


In [ ]:
# Training Logs
! cat ner_logs/MedicalNerApproach*

Name of the selected graph: /content/medical_ner_graphs/blstm_3_200_24_85.pb
Training started - total epochs: 20 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 6347


Epoch 1/20 started, lr: 0.003, dataset size: 6347


Epoch 1/20 - 34.03s - loss: 1926.2507 - avg training loss: 3.028696 - batches: 636
Quality on validation dataset (20.0%), validation examples = 1269
time to finish evaluation: 2.98s
Total validation loss: 220.7569	Avg validation loss: 1.3543
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 1091	 154	 383	 0.8763052	 0.74016285	 0.8025009
B-Disease	 1071	 184	 179	 0.85338646	 0.8568	 0.85508984
tp: 2162 fp: 338 fn: 562 labels: 2
Macro-average	 prec: 0.8648459, rec: 0.79848146, f1: 0.8303397
Micro-average	 prec: 0.8648, rec: 0.79368573, f1: 0.82771826
Quality on test dataset: 
time to finish evaluation: 1.03s
Total test loss: 96.5837	Avg test loss: 1.5091
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 444	 74	 186	 0.85714287	 0.7047619	 0.77351916
B-Dis

As you see above, training was stopped before 20th epoch since there were not improvement.

**Evaluate**

In [ ]:
#from sparknlp_jsl.eval import NerDLMetrics

pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_1))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|366.0|75.0|62.0|428.0|   0.8299|0.8551|0.8423|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8423475258918297|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8423475258918297|
+------------------+

None


In [ ]:
pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|433.0|97.0|94.0|527.0|    0.817|0.8216|0.8193|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8192999053926207|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8192999053926207|
+------------------+

None


## Now let's take a model trained on a different dataset and train on this dataset

In [ ]:
jsl_ner = medical.NerModel.pretrained('ner_jsl','en','clinical/models')

jsl_ner.getClasses()

ner_jsl download started this may take some time.
[OK!]


['O',
 'B-Injury_or_Poisoning',
 'B-Direction',
 'B-Test',
 'I-Route',
 'B-Admission_Discharge',
 'B-Death_Entity',
 'I-Oxygen_Therapy',
 'B-Relationship_Status',
 'I-Drug_BrandName',
 'B-Duration',
 'I-Alcohol',
 'I-Triglycerides',
 'I-Date',
 'B-Hyperlipidemia',
 'B-Respiration',
 'I-Test',
 'B-Birth_Entity',
 'I-VS_Finding',
 'B-Age',
 'I-Vaccine_Name',
 'I-Social_History_Header',
 'B-Labour_Delivery',
 'I-Medical_Device',
 'B-Family_History_Header',
 'B-BMI',
 'I-Fetus_NewBorn',
 'I-BMI',
 'B-Temperature',
 'I-Section_Header',
 'I-Communicable_Disease',
 'I-ImagingFindings',
 'I-Psychological_Condition',
 'I-Obesity',
 'I-Sexually_Active_or_Sexual_Orientation',
 'I-Modifier',
 'B-Alcohol',
 'I-Temperature',
 'I-Vaccine',
 'I-Symptom',
 'I-Pulse',
 'B-Kidney_Disease',
 'B-Oncological',
 'I-EKG_Findings',
 'B-Medical_History_Header',
 'I-Relationship_Status',
 'B-Cerebrovascular_Disease',
 'I-Blood_Pressure',
 'I-Diabetes',
 'B-Oxygen_Therapy',
 'B-O2_Saturation',
 'B-Psychological_C

### Now train a model using this model as base

In [ ]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(2)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('/content/test_2.parquet')\
      .setOutputLogsPath('ner_logs')\
      .setGraphFolder(graph_folder_path)\
      .setPretrainedModelPath("/root/cache_pretrained/ner_jsl_en_4.2.0_3.0_1666181370373")\
      .setOverrideExistingTags(True) # since the tags do not align, set this flag to true

# do hyperparameter by tuning the params above (max epoch, LR, dropout etc.) to get better results
ner_pipeline = nlp.Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [ ]:
# remove the existing logs

! rm -r ./ner_logs

In [ ]:
%%time
ner_jsl_retrained = ner_pipeline.fit(training_data)


TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 10.8 s, sys: 204 ms, total: 11 s
Wall time: 5min


In [ ]:
!cat ./ner_logs/MedicalNerApproach*

Name of the selected graph: pretrained graph
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 103 - training examples: 6347


Epoch 1/2 started, lr: 0.003, dataset size: 6347


Epoch 1/2 - 125.36s - loss: 1363.1628 - avg training loss: 1.7146703 - batches: 795
Quality on test dataset: 
time to finish evaluation: 10.79s
Total test loss: 70.9633	Avg test loss: 1.1088
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 505	 72	 125	 0.87521666	 0.8015873	 0.8367854
B-Disease	 458	 81	 73	 0.8497217	 0.86252356	 0.85607487
tp: 963 fp: 153 fn: 198 labels: 2
Macro-average	 prec: 0.8624692, rec: 0.83205545, f1: 0.8469894
Micro-average	 prec: 0.86290324, rec: 0.82945734, f1: 0.8458498


Epoch 2/2 started, lr: 0.0029850747, dataset size: 6347


Epoch 2/2 - 125.60s - loss: 651.1995 - avg training loss: 0.8191189 - batches: 795
Quality on test dataset: 
time to finish evaluation: 10.58s
Total test loss: 54.0507	Avg test loss: 0.8445
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Di

In [ ]:
#from sparknlp_jsl.eval import NerDLMetrics

pred_df = ner_jsl_retrained.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|442.0|57.0|85.0|527.0|   0.8858|0.8387|0.8616|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8615984405458089|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8615984405458089|
+------------------+

None
